<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/13_Mejor_Estimacion_Decesos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculo de la Mejor Estimación de una póliza de Decesos (Vida Entera)

Nos descargamos la base de datos clonando el repositorio de github:

In [27]:
#!git clone https://github.com/franciscogarate/cdiae/

Importamos las librerias necesarias, así como la tabla de mortalidad y curva libre de riesgo

In [28]:
import pandas as pd
import numpy as np
from cdiae import lx, clr

In [29]:
lx = lx.PASEM2020_Decesos_M_2orden.values.copy() # Tabla PASEM2020 Decesos 2orden Hombres
w = len(lx)                                      # Fin de tabla
edad = 50                                        # Edad en t=0

In [30]:
df = pd.DataFrame(pd.date_range(start='2024-12-31',periods=(w - edad),freq='YE'), columns=['Fecha'])
df['edad'] = edad + df.index
df['t'] = df.index
df['lx'] = df['edad'].apply(lambda x: lx[x])
df.head()

,Fecha,edad,t,lx
0,2024-12-31,50,0,96712.874270
1,2025-12-31,51,1,96371.781940
2,2026-12-31,52,2,95995.343130
3,2027-12-31,53,3,95581.665970
4,2028-12-31,54,4,95128.984820
5,2029-12-31,55,5,94635.458150
6,2030-12-31,56,6,94099.032970
7,2031-12-31,57,7,93517.345640
8,2032-12-31,58,8,92887.802510
9,2033-12-31,59,9,92207.702260


In [31]:
df['qx'] = df['lx'].diff(-1).fillna(0)/df['lx'][0]
df['sum_qx'] = df['qx'].cumsum()
df['px'] = 1 - df['sum_qx']
df.head()

,Fecha,edad,t,lx,qx,sum_qx,px
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581
2,2026-12-31,52,2,95995.34313,0.004277,0.011697,0.988303
3,2027-12-31,53,3,95581.66597,0.004681,0.016377,0.983623
4,2028-12-31,54,4,95128.98482,0.005103,0.021480,0.978520


In [32]:
def incr_capital(capital, t):
  return capital * (1 + 0.015) ** t

df['capital'] = incr_capital(5000, df.t)
df.head()

,Fecha,edad,t,lx,qx,sum_qx,px,capital
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5000.0
2,2026-12-31,52,2,95995.34313,0.004277,0.011697,0.988303,5000.0
3,2027-12-31,53,3,95581.66597,0.004681,0.016377,0.983623,5000.0
4,2028-12-31,54,4,95128.98482,0.005103,0.021480,0.978520,5000.0


In [33]:
df['pagos'] = df['capital'] * df['qx']
df.head()

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5000.0,19.461670
2,2026-12-31,52,2,95995.34313,0.004277,0.011697,0.988303,5000.0,21.386871
3,2027-12-31,53,3,95581.66597,0.004681,0.016377,0.983623,5000.0,23.403355
4,2028-12-31,54,4,95128.98482,0.005103,0.021480,0.978520,5000.0,25.515045


In [34]:
df.pagos.sum()

np.float64(5000.0)

In [35]:
df['caida'] = 0.03
df['polizas'] = (1 - df['caida']).cumprod()
df.head()

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.970000
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5000.0,19.461670,0.03,0.940900
2,2026-12-31,52,2,95995.34313,0.004277,0.011697,0.988303,5000.0,21.386871,0.03,0.912673
3,2027-12-31,53,3,95581.66597,0.004681,0.016377,0.983623,5000.0,23.403355,0.03,0.885293
4,2028-12-31,54,4,95128.98482,0.005103,0.021480,0.978520,5000.0,25.515045,0.03,0.858734


In [36]:
df['polizas_benef'] =  df['px'] * df['polizas']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas,polizas_benef
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5000.0,19.461670,0.03,0.9409,0.933919


In [37]:
df['pagos_prob'] = df['pagos'] * df['polizas_benef']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas,polizas_benef,pagos_prob
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579,17.044921
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5000.0,19.461670,0.03,0.9409,0.933919,18.175629


In [38]:
df['clr'] = df['t'].apply(lambda t: clr[t])
df['factor_desc'] = df.apply(lambda x: 1 / (1 + x.clr) ** (x.t), axis=1)
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas,polizas_benef,pagos_prob,clr,factor_desc
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579,17.044921,0.02236,1.000000
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5000.0,19.461670,0.03,0.9409,0.933919,18.175629,0.02093,0.979499


In [39]:
df['gastos'] = df['capital'] * 0.01
df['gastos_prob'] = df['gastos'] * df['polizas_benef']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas,polizas_benef,pagos_prob,clr,factor_desc,gastos,gastos_prob
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579,17.044921,0.02236,1.000000,50.0,48.328948
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5000.0,19.461670,0.03,0.9409,0.933919,18.175629,0.02093,0.979499,50.0,46.695964


In [40]:
df['salidas_prob'] = df['pagos_prob'] + df['gastos_prob']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas,polizas_benef,pagos_prob,clr,factor_desc,gastos,gastos_prob,salidas_prob
0,2024-12-31,50,0,96712.87427,0.003527,0.003527,0.996473,5000.0,17.634277,0.03,0.9700,0.966579,17.044921,0.02236,1.000000,50.0,48.328948,65.373869
1,2025-12-31,51,1,96371.78194,0.003892,0.007419,0.992581,5000.0,19.461670,0.03,0.9409,0.933919,18.175629,0.02093,0.979499,50.0,46.695964,64.871593


In [41]:
print(f'BEL: {(df.salidas_prob @ df.factor_desc):,.2f}')

BEL: 1,468.36
